# 프로젝트 설명

In [2]:
import sys
sys.path.append("..") 

In [ ]:
from src.utility import *
project_name = "dual_attention"
createfolder("./data")
createfolder("./model")
createfolder("./result")

# Data load

In [17]:
num_data = 14
data_list = []

for i in range(num_data):
    data_load = loadfile('./data/pre_data', f'Series {i+1}','csv')
    data_list.append(data_load)

csv file is loaded from ./data/pre_data/Series 1.csv
csv file is loaded from ./data/pre_data/Series 2.csv
csv file is loaded from ./data/pre_data/Series 3.csv
csv file is loaded from ./data/pre_data/Series 4.csv
csv file is loaded from ./data/pre_data/Series 5.csv
csv file is loaded from ./data/pre_data/Series 6.csv
csv file is loaded from ./data/pre_data/Series 7.csv
csv file is loaded from ./data/pre_data/Series 8.csv
csv file is loaded from ./data/pre_data/Series 9.csv
csv file is loaded from ./data/pre_data/Series 10.csv
csv file is loaded from ./data/pre_data/Series 11.csv
csv file is loaded from ./data/pre_data/Series 12.csv
csv file is loaded from ./data/pre_data/Series 13.csv
csv file is loaded from ./data/pre_data/Series 14.csv


## Data analysis

In [19]:
sum = 0
for i in range(num_data):
    print(f"Series {i+1}: {data_list[i].shape}")
    sum += len(data_list[i])
    
print(f"Total number of data: {sum}")

Series 1: (843, 14)
Series 2: (1959, 14)
Series 3: (575, 14)
Series 4: (569, 14)
Series 5: (863, 14)
Series 6: (742, 14)
Series 7: (2414, 14)
Series 8: (645, 14)
Series 9: (927, 14)
Series 10: (980, 14)
Series 11: (2425, 14)
Series 12: (857, 14)
Series 13: (517, 14)
Series 14: (638, 14)
Total number of data: 14954


In [20]:
data_list[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 843 entries, 0 to 842
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   FT-3061-2  843 non-null    float64
 1   FT-3061-3  843 non-null    float64
 2   FT-3061-4  843 non-null    float64
 3   FT-3062-1  843 non-null    float64
 4   TT-3061-1  843 non-null    float64
 5   PT-3061-2  843 non-null    float64
 6   TT-3061-2  843 non-null    float64
 7   LT-3062-1  843 non-null    float64
 8   TT-3061-4  843 non-null    float64
 9   TT-3061-6  843 non-null    float64
 10  PT-3061-3  843 non-null    float64
 11  TT-3061-3  843 non-null    float64
 12  TT-3061-5  843 non-null    float64
 13  LT-3061-2  843 non-null    float64
dtypes: float64(14)
memory usage: 92.3 KB


In [21]:
data_list[0].describe()

,FT-3061-2,FT-3061-3,FT-3061-4,FT-3062-1,TT-3061-1,PT-3061-2,TT-3061-2,LT-3062-1,TT-3061-4,TT-3061-6,PT-3061-3,TT-3061-3,TT-3061-5,LT-3061-2
count,843.000000,843.000000,843.000000,843.000000,843.000000,843.000000,843.000000,843.000000,843.000000,843.000000,843.000000,843.000000,843.000000,843.000000
mean,66.631501,115.696069,45.523176,40.923389,51.853239,-0.959493,41.378565,29.015488,121.956926,114.102031,-0.783211,84.305516,42.360885,41.659122
std,8.600889,2.437222,5.036386,6.272298,0.559134,0.000868,0.155463,0.645218,2.053312,1.668485,0.019893,1.725717,0.708202,10.023022
min,49.841917,110.153779,37.600000,25.262479,51.326837,-0.961135,41.050196,27.072900,117.100000,109.441035,-0.831101,80.070640,41.525176,27.984438
25%,62.776486,113.815243,41.971216,37.034912,51.443183,-0.960316,41.245471,28.488705,120.507010,114.261800,-0.788143,83.119591,41.872815,33.196159
50%,69.989736,115.471903,43.328756,39.402951,51.603292,-0.959460,41.401961,29.043290,122.069817,114.740411,-0.781250,84.221185,42.110989,39.475913
75%,72.232714,117.390341,48.526162,43.769661,52.214066,-0.958772,41.489596,29.616456,123.842088,114.921381,-0.768168,85.631964,42.747933,50.919665
max,77.324448,122.415228,54.570497,55.489132,53.063916,-0.957798,41.690921,30.227521,124.595467,115.824917,-0.757128,87.190709,45.905564,61.854325


# Data preparation

In [22]:
from src import rnn
import numpy as np

In [23]:
history_length = 60
future_length = 20
step = 1

history_var = ["FT-3061-2", "FT-3061-3", "FT-3061-4", "FT-3062-1", "TT-3061-3", "TT-3061-4", "LT-3061-2"]
future_var = ["TT-3061-3", "TT-3061-4", "LT-3061-2"]

history_series = []
future_series = []

for i in range(num_data):
    history, future = rnn.data2series(data_list[i], history_length, history_var, future_length, future_var,
                                      step, start_index=0, end_index=None)
    if not i:
        history_series = history
        future_series = future
    else:
        history_series = np.concatenate([history_series, history], axis=0)
        future_series = np.concatenate([future_series, future], axis=0)

In [26]:
print(f"history series: {history_series.shape}",
      f"future series: {future_series.shape}",
      sep='\n')

history series: (13834, 60, 7)
future series: (13834, 20, 3)
